# Parse the diamond outputs for gene abundances analysis (metagenomic and metatranscriptomic)

In [1]:
import pandas as pd
import os
import glob
from Bio import SeqIO

In [2]:
gene='nifH'

In [3]:
def diamond_parse(diamond_path, column1='perc_id', cutoff1= 95, column2='length', cutoff2=30):
    """
    This function is directed to a diamond_path, which refers to all the diamond.out files in a folder 
    We read it in, adding in column names, and filter the columns perc_id and length based on designated thresholds.
    Here we use 95% perc_id and 30 amino acids in length (200 residues/3 = 66 amino acids *50%)
    Then we groupby the subject_sequence in order to get the number of reads that mapped onto a particular protein id (k-id)
    We normalize these counts to account for "multimapping"reads, that is, when our ERR query_sequence mapped to several subject_sequences, or protein k-ids, given highly conserved regions.
    These .csv files are output with basename of the ERR file and "norm_subject_counts"
    """
    diamond_files=glob.glob(diamond_path)
    for filename in diamond_files:
        filter_df=[]
        open(filename)
        base=os.path.basename(filename).replace(".diamond.out","")
        if os.stat(filename).st_size == 0:
            pass
        df=pd.read_csv(filename, sep='\t', header=None)
        df.columns=['query_sequence','subject_sequence','perc_id','length','mismatch','gap_open','query_start','query_end','subject_start','subject_end','e_value','bit_score']
        filter_df=df[(df[column1]>=cutoff1) & (df[column2]>=cutoff2)]
        count_map=filter_df.groupby(['query_sequence','subject_sequence'])['perc_id'].count().unstack().fillna(0) #gives us 1 or 0 presence/absence for each k-id or subject (along x axis) for each ERR or query_sequence (y-axis)
        count_map_sum=count_map.sum(axis=1)#flips orientation and sums 
        df_norm_counts= pd.DataFrame(count_map.divide(count_map_sum, axis=0).sum(axis=0)) #divide to get our factor for normalizing
        df_norm_counts.columns= [os.path.basename(base).replace(".diamond.out","")]
        df_norm_counts.to_csv(os.path.join(output_folder, base+"_norm_subject_counts.csv")) #our second output to .csv, for normalizing by the number of protein k-ids an ERR query_sequence matched to 
            
        
        

In [4]:
output_folder = os.path.join("/vortexfs1/omics/alexander/lblum/tara_gene_finder/results/gene_abundances/diamond_mapping", gene,'metaG_abundance_outputs') 
#output folder is for metaG abundance data

metaG_diamond_folder= os.path.join('/vortexfs1/omics/alexander/lblum/tara_gene_finder/results/gene_abundances/diamond_mapping', gene, 'PRJEB4352/*')
#note the two PRJ files, 4352 is metagenome and 6609 is metatranscriptomic

diamond_parse(metaG_diamond_folder)

ValueError: Length mismatch: Expected axis has 1 elements, new values have 12 elements

In [5]:
output_folder= os.path.join('/vortexfs1/omics/alexander/lblum/tara_gene_finder/results/gene_abundances/diamond_mapping',gene,'metaT_abundance_outputs')
#run function with output for metatranscritomic data

metaT_diamond_folder= os.path.join('/vortexfs1/omics/alexander/lblum/tara_gene_finder/results/gene_abundances/diamond_mapping', gene, 'PRJEB6609/*') 
#location of diamond outputs for this gene for metatranscriptomic data

diamond_parse(metaT_diamond_folder)

KeyboardInterrupt: 

In [6]:
#read in all the intermediate files for metaG abundance data and concatenate
from functools import reduce
metaG_abundance_path = os.path.join('/vortexfs1/omics/alexander/lblum/tara_gene_finder/results/gene_abundances/diamond_mapping',gene, 'metaG_abundance_outputs/')
filenames = glob.glob(metaG_abundance_path + "/*.csv")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))

# Concatenate all data into one DataFrame
metaG_abundance = reduce(lambda x, y: pd.merge(x, y, on='subject_sequence', how='outer'), dfs)
metaG_abundance=metaG_abundance.set_index('subject_sequence')

In [7]:
#read in all the intermediate files for metaT abundance data and concatenate
from functools import reduce
metaT_abundance_path = os.path.join('/vortexfs1/omics/alexander/lblum/tara_gene_finder/results/gene_abundances/diamond_mapping',gene,'metaT_abundance_outputs/')
filenames = glob.glob(metaT_abundance_path + "/*.csv")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))
    
# Concatenate all data into one DataFrame
metaT_abundance = reduce(lambda x, y: pd.merge(x, y, on='subject_sequence', how='outer'), dfs)
metaT_abundance=metaT_abundance.set_index('subject_sequence')

In [8]:
metaG_abundance=metaG_abundance.fillna(0)
metaG_abundance.to_csv(os.path.join('/vortexfs1/omics/alexander/lblum/tara_gene_finder/results/gene_abundances/diamond_mapping',gene, gene+'_metaG_PRJEB4352_95_raw_abundance_table.csv'))

In [9]:
metaG_95_raw= pd.read_csv(os.path.join('/vortexfs1/omics/alexander/lblum/tara_gene_finder/results/gene_abundances/diamond_mapping',gene,gene+'_metaG_PRJEB4352_95_raw_abundance_table.csv'), index_col='subject_sequence')

In [6]:
metaG_wenv=pd.read_csv('/vortexfs1/omics/alexander/lblum/tara_gene_finder/results/gene_abundances/PANGEA_Merged_TARA.csv', index_col='run_accession')

In [11]:
metaG_RPM=metaG_95_raw/(metaG_wenv.read_count/1000000)
metaG_RPM_dropna=metaG_RPM.dropna(how='any', axis=1)  #drop columns that were na (meaning we didn't have that ERR)

#get our sequence lengths for normalization (RPKM) from the fasta file with the sequences clustered at 98% similar
sequence_lengths = []

for record in SeqIO.parse(os.path.join(gene+'_clust_98'), "fasta") :
    subject_sequence = record.id
    length = len(record.seq)
    sequence_lengths.append((subject_sequence, length))
    
sequence_lengths_df=pd.DataFrame(sequence_lengths, columns=('subject_sequence','length')) 

#here we use our sequence_lengths_df to normalize by gene length giving us RPKM for abundances. 
metaG_RPM_with_length= metaG_RPM_dropna.merge(sequence_lengths_df, left_on='subject_sequence', right_on='subject_sequence').set_index('subject_sequence')
RPKM_metaG= metaG_RPM_with_length.iloc[:, 1:-1].divide(metaG_RPM_with_length.iloc[:,-1], axis = 'rows')
RPKM_metaG.to_csv(os.path.join('/vortexfs1/omics/alexander/lblum/tara_gene_finder/results/gene_abundances/diamond_mapping',gene, gene+'_RPKM_metaG_PRJEB4352_95_abundance_table.csv'))

In [7]:
#metatranscriptomic metadata
metaT_wenv=pd.read_csv('/vortexfs1/omics/alexander/lblum/tara_gene_finder/results/gene_abundances/PANGEA_Merged_TARA_metaT.csv', index_col='run_accession')

In [13]:
metaT_abundance=metaT_abundance.fillna(0)
metaT_abundance.to_csv(os.path.join('/vortexfs1/omics/alexander/lblum/tara_gene_finder/results/gene_abundances/diamond_mapping', gene, gene+'_metaT_PRJEB6609_95_raw_abundance_table.csv'))

In [14]:
metaT_95_raw= pd.read_csv(os.path.join('/vortexfs1/omics/alexander/lblum/tara_gene_finder/results/gene_abundances/diamond_mapping', gene, gene+'_metaT_PRJEB6609_95_raw_abundance_table.csv'), index_col='subject_sequence')

In [15]:
metaT_RPM=metaT_95_raw/(metaT_wenv.read_count/1000000)
metaT_RPM_dropna=metaT_RPM.dropna(how='any', axis=1)  #drop columns that were na (meaning we didn't have that ERR)

#get our sequence lengths for normalization (RPKM) from the fasta file withthe clustered 98 seqs

sequence_lengths = []

for record in SeqIO.parse(os.path.join(gene+'_clust_98'), "fasta") :
    subject_sequence = record.id
    length = len(record.seq)
    sequence_lengths.append((subject_sequence, length))
    
sequence_lengths_df=pd.DataFrame(sequence_lengths, columns=('subject_sequence','length')) 

#here we use our sequence_lengths_df to normalize by gene length giving us RPKM for abundances.
metaT_RPM_with_length= metaT_RPM_dropna.merge(sequence_lengths_df, left_on='subject_sequence', right_on='subject_sequence').set_index('subject_sequence')
RPKM_metaT= metaT_RPM_with_length.iloc[:, 1:-1].divide(metaT_RPM_with_length.iloc[:,-1], axis = 'rows')
RPKM_metaT.to_csv(os.path.join('/vortexfs1/omics/alexander/lblum/tara_gene_finder/results/gene_abundances/diamond_mapping', gene, gene+'_RPKM_metaT_PRJEB6609_95_abundance_table.csv'))


In [18]:
#creating a df of our normalized counts by sample and some environmental data for metaG
metaG_abundance_sum=RPKM_metaG.sum()
metaG_abundance_frame=metaG_abundance_sum.to_frame(name='gene_abundance')
metaG_abundance_wenv=metaG_abundance_frame.merge(metaG_wenv, left_index=True, right_on='run_accession')
metaG_abundance_wenv["Depth"]=metaG_abundance_wenv['Sample material'].str.split("_").str[2]
metaG_abundance_wenv["Size_fraction"]=metaG_abundance_wenv['Sample material'].str.split("_").str[3]
metaG_abundance_wenv["Size_fraction_group"]=metaG_abundance_wenv['Size_fraction'].replace('0.8->','0.8-5').replace('0.8-3','0.8-5').replace('0.8-20','5-20').replace('3->','0.8-5').replace('0.22-3','0.8-5')
metaG_abundance_wenv.to_csv(os.path.join('/vortexfs1/omics/alexander/lblum/tara_gene_finder/results/gene_abundances/diamond_mapping',gene, gene+'_metaG_abundance_wenv_table.csv'))

In [19]:
#creating a df of our normalized counts by sample and some environmental data for metaT

metaT_abundance_sum=RPKM_metaT.sum()
metaT_abundance_frame=metaT_abundance_sum.to_frame(name='gene_abundance')
metaT_abundance_wenv=metaT_abundance_frame.merge(metaT_wenv, left_index=True, right_on='run_accession')
metaT_abundance_wenv["Depth"]=metaT_abundance_wenv['Sample material'].str.split("_").str[2]
metaT_abundance_wenv["Size_fraction"]=metaT_abundance_wenv['Sample material'].str.split("_").str[3]
metaT_abundance_wenv["Size_fraction_group"]=metaT_abundance_wenv['Size_fraction'].replace('0.8->','0.8-5').replace('0.8-3','0.8-5').replace('0.8-20','5-20').replace('3->','0.8-5').replace('0.22-3','0.8-5')
metaT_abundance_wenv.to_csv(os.path.join('/vortexfs1/omics/alexander/lblum/tara_gene_finder/results/gene_abundances/diamond_mapping',gene, gene+'_metaT_abundance_wenv_table.csv'))